# Loading Libraries

Connect to Earth engine, import packages.

In [ ]:
 import ee
!pip install geopandas geemap geehydro
import geemap
import folium
import os
import geehydro

try:
        ee.Initialize()
except Exception as e:
        ee.Authenticate()
        ee.Initialize(project='ccri-chla-modeling')

# from google.colab import drive
# drive.mount('/content/drive')
# out_dir = os.path.expanduser('/content/drive/test')


Import assets from gee

In [ ]:
## shapefile (STORETID) table as csv
from google.colab import files
uploaded = files.upload()
import pandas as pd
import io

#assets from GEE
UniquePts = ee.FeatureCollection("users/greeneji/MiCorp_Secchi_4ha_dissolve")

STORETIDs = pd.read_csv(io.BytesIO(uploaded['MiCorp_Secchi_4ha_dissolve.csv']))
STORETIDs1 = STORETIDs.iloc[0:, 1].dropna().to_list()

Saving MiCorp_Secchi_4ha_dissolve.csv to MiCorp_Secchi_4ha_dissolve.csv




# MAIN Atmospheric Correction

Page, B.P., Olmanson, L.G. and Mishra, D.R., 2019. A harmonized image processing workflow using Sentinel-2/MSI and Landsat-8/OLI for mapping water clarity in optically variable lake systems. Remote Sensing of Environment, 231, p.111284.

https://github.com/Nateme16/geo-aquawatch-water-quality/blob/main/Atmospheric%20corrections/main_L8L9.ipynb

In [ ]:
# MAIN Atmospheric Correction
pi = ee.Image(3.141592);


def MAIN_S2A(img):

    # msi bands
    bands = ['B1','B2','B3','B4','B5','B6','B7', 'B8', 'B8A', 'B11', 'B12'];

    # rescale
    rescale = img.select(bands).divide(10000)

    # tile footprint
    footprint = rescale.geometry()

    # dem
    DEM = ee.Image('USGS/SRTMGL1_003').clip(footprint);

    # ozone
    DU = ee.Image(300);
    #ee.Image(ozone.filterDate(startDate,endDate).filterBounds(footprint).mean());

    # Julian Day
    imgDate = ee.Date(img.get('system:time_start'));
    FOY = ee.Date.fromYMD(imgDate.get('year'),1,1);
    JD = imgDate.difference(FOY,'day').int().add(1);

    # earth-sun distance
    myCos = ((ee.Image(0.0172).multiply(ee.Image(JD).subtract(ee.Image(2)))).cos()).pow(2)
    cosd = myCos.multiply(pi.divide(ee.Image(180))).cos();
    d = ee.Image(1).subtract(ee.Image(0.01673)).multiply(cosd).clip(footprint)

    # sun azimuth
    SunAz = ee.Image.constant(img.get('MEAN_SOLAR_AZIMUTH_ANGLE')).clip(footprint);

    # sun zenith
    SunZe = ee.Image.constant(img.get('MEAN_SOLAR_ZENITH_ANGLE')).clip(footprint);
    cosdSunZe = SunZe.multiply(pi.divide(ee.Image(180))).cos(); # in degrees
    sindSunZe = SunZe.multiply(pi.divide(ee.Image(180))).sin(); # in degrees

    # sat zenith
    SatZe = ee.Image.constant(img.get('MEAN_INCIDENCE_ZENITH_ANGLE_B5')).clip(footprint);
    cosdSatZe = (SatZe).multiply(pi.divide(ee.Image(180))).cos();
    sindSatZe = (SatZe).multiply(pi.divide(ee.Image(180))).sin();

    # sat azimuth
    SatAz = ee.Image.constant(img.get('MEAN_INCIDENCE_AZIMUTH_ANGLE_B5')).clip(footprint);

    # relative azimuth
    RelAz = SatAz.subtract(SunAz);
    cosdRelAz = RelAz.multiply(pi.divide(ee.Image(180))).cos();

    # Pressure
    P = ee.Image(101325).multiply(ee.Image(1).subtract(ee.Image(0.0000225577).multiply(DEM)).pow(5.25588)).multiply(0.01)
    Po = ee.Image(1013.25);

    # esun
    ESUN = ee.Image(ee.Array([ee.Image(img.get('SOLAR_IRRADIANCE_B1')),
                      ee.Image(img.get('SOLAR_IRRADIANCE_B2')),
                      ee.Image(img.get('SOLAR_IRRADIANCE_B3')),
                      ee.Image(img.get('SOLAR_IRRADIANCE_B4')),
                      ee.Image(img.get('SOLAR_IRRADIANCE_B5')),
                      ee.Image(img.get('SOLAR_IRRADIANCE_B6')),
                      ee.Image(img.get('SOLAR_IRRADIANCE_B7')),
                      ee.Image(img.get('SOLAR_IRRADIANCE_B8')),
                      ee.Image(img.get('SOLAR_IRRADIANCE_B8A')),
                      ee.Image(img.get('SOLAR_IRRADIANCE_B11')),
                      ee.Image(img.get('SOLAR_IRRADIANCE_B12'))]
                      )).toArray().toArray(1);

    ESUN = ESUN.multiply(ee.Image(1))

    ESUNImg = ESUN.arrayProject([0]).arrayFlatten([bands]);

    # create empty array for the images
    imgArr = rescale.select(bands).toArray().toArray(1);

    # pTOA to Ltoa
    Ltoa = imgArr.multiply(ESUN).multiply(cosdSunZe).divide(pi.multiply(d.pow(2)));

    # band centers
    bandCenter = ee.Image(444).divide(1000).addBands(ee.Image(496).divide(1000)).addBands(ee.Image(560).divide(1000)).addBands(ee.Image(664).divide(1000)).addBands(ee.Image(704).divide(1000)).addBands(ee.Image(740).divide(1000)).addBands(ee.Image(782).divide(1000)).addBands(ee.Image(835).divide(1000)).addBands(ee.Image(865).divide(1000)).addBands(ee.Image(1613).divide(1000)).addBands(ee.Image(2202).divide(1000)).toArray().toArray(1);

    # ozone coefficients
    koz = ee.Image(0.0040).addBands(ee.Image(0.0244)).addBands(ee.Image(0.1052)).addBands(ee.Image(0.0516)).addBands(ee.Image(0.0208)).addBands(ee.Image(0.0112)).addBands(ee.Image(0.0079)).addBands(ee.Image(0.0021)).addBands(ee.Image(0.0019))                          .addBands(ee.Image(0)).addBands(ee.Image(0)).toArray().toArray(1);

    # Calculate ozone optical thickness
    Toz = koz.multiply(DU).divide(ee.Image(1000));

    # Calculate TOA radiance in the absense of ozone
    Lt = Ltoa.multiply(((Toz)).multiply((ee.Image(1).divide(cosdSunZe)).add(ee.Image(1).divide(cosdSatZe))).exp());

    # Rayleigh optical thickness
    Tr = (P.divide(Po)).multiply(ee.Image(0.008569).multiply(bandCenter.pow(-4))).multiply((ee.Image(1).add(ee.Image(0.0113).multiply(bandCenter.pow(-2))).add(ee.Image(0.00013).multiply(bandCenter.pow(-4)))));

    # Specular reflection (s- and p- polarization states)
    theta_V = ee.Image(0.0000000001);
    sin_theta_j = sindSunZe.divide(ee.Image(1.333));

    theta_j = sin_theta_j.asin().multiply(ee.Image(180).divide(pi));

    theta_SZ = SunZe;

    R_theta_SZ_s = (((theta_SZ.multiply(pi.divide(ee.Image(180)))).subtract(theta_j.multiply(pi.divide(ee.Image(180))))).sin().pow(2)).divide((((theta_SZ.multiply(pi.divide(ee.Image(180)))).add(theta_j.multiply(pi.divide(ee.Image(180))))).sin().pow(2)));

    R_theta_V_s = ee.Image(0.0000000001);

    R_theta_SZ_p = (((theta_SZ.multiply(pi.divide(180))).subtract(theta_j.multiply(pi.divide(180)))).tan().pow(2)).divide((((theta_SZ.multiply(pi.divide(180))).add(theta_j.multiply(pi.divide(180)))).tan().pow(2)));

    R_theta_V_p = ee.Image(0.0000000001);

    R_theta_SZ = ee.Image(0.5).multiply(R_theta_SZ_s.add(R_theta_SZ_p));

    R_theta_V = ee.Image(0.5).multiply(R_theta_V_s.add(R_theta_V_p));

    # Sun-sensor geometry
    theta_neg = ((cosdSunZe.multiply(ee.Image(-1))).multiply(cosdSatZe)).subtract((sindSunZe).multiply(sindSatZe).multiply(cosdRelAz));

    theta_neg_inv = theta_neg.acos().multiply(ee.Image(180).divide(pi));

    theta_pos = (cosdSunZe.multiply(cosdSatZe)).subtract(sindSunZe.multiply(sindSatZe).multiply(cosdRelAz));

    theta_pos_inv = theta_pos.acos().multiply(ee.Image(180).divide(pi));

    cosd_tni = theta_neg_inv.multiply(pi.divide(180)).cos(); # in degrees

    cosd_tpi = theta_pos_inv.multiply(pi.divide(180)).cos(); # in degrees

    Pr_neg = ee.Image(0.75).multiply((ee.Image(1).add(cosd_tni.pow(2))));

    Pr_pos = ee.Image(0.75).multiply((ee.Image(1).add(cosd_tpi.pow(2))));

    # Rayleigh scattering phase function
    Pr = Pr_neg.add((R_theta_SZ.add(R_theta_V)).multiply(Pr_pos));

    # rayleigh radiance contribution
    denom = ee.Image(4).multiply(pi).multiply(cosdSatZe);
    Lr = (ESUN.multiply(Tr)).multiply(Pr.divide(denom));

    # rayleigh corrected radiance
    Lrc = Lt.subtract(Lr);
    LrcImg = Lrc.arrayProject([0]).arrayFlatten([bands]);
    prcImg = (Lrc.multiply(pi).multiply(d.pow(2)).divide(ESUN.multiply(cosdSunZe)));
    prcImg = prcImg.arrayProject([0]).arrayFlatten([bands]);


    # Aerosol Correction

    # Bands in nm
    bands_nm = ee.Image(444).addBands(ee.Image(496)).addBands(ee.Image(560)).addBands(ee.Image(664)).addBands(ee.Image(703)).addBands(ee.Image(740)).addBands(ee.Image(782)).addBands(ee.Image(835)).addBands(ee.Image(865)).addBands(ee.Image(0)) .addBands(ee.Image(0)).toArray().toArray(1);

    # Lam in SWIR bands
    Lam_10 = LrcImg.select('B11')
    Lam_11 = LrcImg.select('B12')

    # Calculate aerosol type
    eps = ((((Lam_11).divide(ESUNImg.select('B12'))).log()).subtract(((Lam_10).divide(ESUNImg.select('B11'))).log())).divide(ee.Image(2190).subtract(ee.Image(1610)));

    # Calculate multiple scattering of aerosols for each band
    Lam = (Lam_11).multiply(((ESUN).divide(ESUNImg.select('B12')))).multiply((eps.multiply(ee.Image(-1))).multiply((bands_nm.divide(ee.Image(2190)))).exp());

    # diffuse transmittance
    trans = Tr.multiply(ee.Image(-1)).divide(ee.Image(2)).multiply(ee.Image(1).divide(cosdSatZe)).exp();

    # Compute water-leaving radiance
    Lw = Lrc.subtract(Lam).divide(trans);

    # water-leaving reflectance
    pw = (Lw.multiply(pi).multiply(d.pow(2)).divide(ESUN.multiply(cosdSunZe)));

    # remote sensing reflectance
    Rrs_S2A = (pw.divide(pi).arrayProject([0]).arrayFlatten([bands]).slice(0,9));

    # set negatives to null
    Rrs_S2A = Rrs_S2A.updateMask(Rrs_S2A.select('B1').gt(0)).multiply(mask)

    return Rrs_S2A.set('system:time_start',img.get('system:time_start'))

In [ ]:
# MAIN for S2B
def MAIN_S2B(img):

    # msi bands
    bands = ['B1','B2','B3','B4','B5','B6','B7', 'B8', 'B8A', 'B11', 'B12'];

    # rescale
    rescale = img.select(bands).divide(10000)

    # tile footprint
    footprint = rescale.geometry()

    # dem
    DEM = ee.Image('USGS/SRTMGL1_003').clip(footprint);

    # ozone
    DU = ee.Image(300);

    # Julian Day
    imgDate = ee.Date(img.get('system:time_start'));
    FOY = ee.Date.fromYMD(imgDate.get('year'),1,1);
    JD = imgDate.difference(FOY,'day').int().add(1);

    # earth-sun distance
    myCos = ((ee.Image(0.0172).multiply(ee.Image(JD).subtract(ee.Image(2)))).cos()).pow(2)
    cosd = myCos.multiply(pi.divide(ee.Image(180))).cos();
    d = ee.Image(1).subtract(ee.Image(0.01673)).multiply(cosd).clip(footprint)

    # sun azimuth
    SunAz = ee.Image.constant(img.get('MEAN_SOLAR_AZIMUTH_ANGLE')).clip(footprint);

    # sun zenith
    SunZe = ee.Image.constant(img.get('MEAN_SOLAR_ZENITH_ANGLE')).clip(footprint);
    cosdSunZe = SunZe.multiply(pi.divide(ee.Image(180))).cos(); # in degrees
    sindSunZe = SunZe.multiply(pi.divide(ee.Image(180))).sin(); # in degrees

    # sat zenith
    SatZe = ee.Image.constant(img.get('MEAN_INCIDENCE_ZENITH_ANGLE_B5')).clip(footprint);
    cosdSatZe = (SatZe).multiply(pi.divide(ee.Image(180))).cos();
    sindSatZe = (SatZe).multiply(pi.divide(ee.Image(180))).sin();

    # sat azimuth
    SatAz = ee.Image.constant(img.get('MEAN_INCIDENCE_AZIMUTH_ANGLE_B5')).clip(footprint);

    # relative azimuth
    RelAz = SatAz.subtract(SunAz);
    cosdRelAz = RelAz.multiply(pi.divide(ee.Image(180))).cos();

    # Pressure
    P = ee.Image(101325).multiply(ee.Image(1).subtract(ee.Image(0.0000225577).multiply(DEM)).pow(5.25588)).multiply(0.01)
    Po = ee.Image(1013.25);

    # esun
    ESUN = ee.Image(ee.Array([ee.Image(img.get('SOLAR_IRRADIANCE_B1')),
                      ee.Image(img.get('SOLAR_IRRADIANCE_B2')),
                      ee.Image(img.get('SOLAR_IRRADIANCE_B3')),
                      ee.Image(img.get('SOLAR_IRRADIANCE_B4')),
                      ee.Image(img.get('SOLAR_IRRADIANCE_B5')),
                      ee.Image(img.get('SOLAR_IRRADIANCE_B6')),
                      ee.Image(img.get('SOLAR_IRRADIANCE_B7')),
                      ee.Image(img.get('SOLAR_IRRADIANCE_B8')),
                      ee.Image(img.get('SOLAR_IRRADIANCE_B8A')),
                      ee.Image(img.get('SOLAR_IRRADIANCE_B11')),
                      ee.Image(img.get('SOLAR_IRRADIANCE_B12'))]
                      )).toArray().toArray(1);

    ESUN = ESUN.multiply(ee.Image(1))

    ESUNImg = ESUN.arrayProject([0]).arrayFlatten([bands]);

    # create empty array for the images
    imgArr = rescale.select(bands).toArray().toArray(1);

    # pTOA to Ltoa
    Ltoa = imgArr.multiply(ESUN).multiply(cosdSunZe).divide(pi.multiply(d.pow(2)));

    # band centers
    bandCenter = ee.Image(442).divide(1000).addBands(ee.Image(492).divide(1000)).addBands(ee.Image(559).divide(1000)).addBands(ee.Image(665).divide(1000)).addBands(ee.Image(703).divide(1000)).addBands(ee.Image(739).divide(1000)).addBands(ee.Image(779).divide(1000)).addBands(ee.Image(833).divide(1000)).addBands(ee.Image(864).divide(1000)).addBands(ee.Image(1610).divide(1000)).addBands(ee.Image(2185).divide(1000)).toArray().toArray(1);

    # ozone coefficients
    koz = ee.Image(0.0037).addBands(ee.Image(0.0223)).addBands(ee.Image(0.1027)).addBands(ee.Image(0.0505)).addBands(ee.Image(0.0212)).addBands(ee.Image(0.0112)).addBands(ee.Image(0.0085)).addBands(ee.Image(0.0022)).addBands(ee.Image(0.0021))                          .addBands(ee.Image(0)).addBands(ee.Image(0)).toArray().toArray(1);

    # Calculate ozone optical thickness
    Toz = koz.multiply(DU).divide(ee.Image(1000));

    # Calculate TOA radiance in the absense of ozone
    Lt = Ltoa.multiply(((Toz)).multiply((ee.Image(1).divide(cosdSunZe)).add(ee.Image(1).divide(cosdSatZe))).exp());

    # Rayleigh optical thickness
    Tr = (P.divide(Po)).multiply(ee.Image(0.008569).multiply(bandCenter.pow(-4))).multiply((ee.Image(1).add(ee.Image(0.0113).multiply(bandCenter.pow(-2))).add(ee.Image(0.00013).multiply(bandCenter.pow(-4)))));

    # Specular reflection (s- and p- polarization states)
    theta_V = ee.Image(0.0000000001);
    sin_theta_j = sindSunZe.divide(ee.Image(1.333));

    theta_j = sin_theta_j.asin().multiply(ee.Image(180).divide(pi));

    theta_SZ = SunZe;

    R_theta_SZ_s = (((theta_SZ.multiply(pi.divide(ee.Image(180)))).subtract(theta_j.multiply(pi.divide(ee.Image(180))))).sin().pow(2)).divide((((theta_SZ.multiply(pi.divide(ee.Image(180)))).add(theta_j.multiply(pi.divide(ee.Image(180))))).sin().pow(2)));

    R_theta_V_s = ee.Image(0.0000000001);

    R_theta_SZ_p = (((theta_SZ.multiply(pi.divide(180))).subtract(theta_j.multiply(pi.divide(180)))).tan().pow(2)).divide((((theta_SZ.multiply(pi.divide(180))).add(theta_j.multiply(pi.divide(180)))).tan().pow(2)));

    R_theta_V_p = ee.Image(0.0000000001);

    R_theta_SZ = ee.Image(0.5).multiply(R_theta_SZ_s.add(R_theta_SZ_p));

    R_theta_V = ee.Image(0.5).multiply(R_theta_V_s.add(R_theta_V_p));

    # Sun-sensor geometry
    theta_neg = ((cosdSunZe.multiply(ee.Image(-1))).multiply(cosdSatZe)).subtract((sindSunZe).multiply(sindSatZe).multiply(cosdRelAz));

    theta_neg_inv = theta_neg.acos().multiply(ee.Image(180).divide(pi));

    theta_pos = (cosdSunZe.multiply(cosdSatZe)).subtract(sindSunZe.multiply(sindSatZe).multiply(cosdRelAz));

    theta_pos_inv = theta_pos.acos().multiply(ee.Image(180).divide(pi));

    cosd_tni = theta_neg_inv.multiply(pi.divide(180)).cos(); # in degrees

    cosd_tpi = theta_pos_inv.multiply(pi.divide(180)).cos(); # in degrees

    Pr_neg = ee.Image(0.75).multiply((ee.Image(1).add(cosd_tni.pow(2))));

    Pr_pos = ee.Image(0.75).multiply((ee.Image(1).add(cosd_tpi.pow(2))));

    # Rayleigh scattering phase function
    Pr = Pr_neg.add((R_theta_SZ.add(R_theta_V)).multiply(Pr_pos));

    # rayleigh radiance contribution
    denom = ee.Image(4).multiply(pi).multiply(cosdSatZe);
    Lr = (ESUN.multiply(Tr)).multiply(Pr.divide(denom));

    # rayleigh corrected radiance
    Lrc = Lt.subtract(Lr);
    LrcImg = Lrc.arrayProject([0]).arrayFlatten([bands]);
    prcImg = (Lrc.multiply(pi).multiply(d.pow(2)).divide(ESUN.multiply(cosdSunZe)));
    prcImg = prcImg.arrayProject([0]).arrayFlatten([bands]);

    # Aerosol Correction

    # Bands in nm
    bands_nm = ee.Image(442).addBands(ee.Image(492)).addBands(ee.Image(559)).addBands(ee.Image(665)).addBands(ee.Image(703)).addBands(ee.Image(739)).addBands(ee.Image(779)).addBands(ee.Image(833)).addBands(ee.Image(864))                            .addBands(ee.Image(0)) .addBands(ee.Image(0)).toArray().toArray(1);

    # Lam in SWIR bands
    Lam_10 = LrcImg.select('B11'); # = 0
    Lam_11 = LrcImg.select('B12'); # = 0

    # Calculate aerosol type
    eps = ((((Lam_11).divide(ESUNImg.select('B12'))).log()).subtract(((Lam_10).divide(ESUNImg.select('B11'))).log())).divide(ee.Image(2190).subtract(ee.Image(1610)));

    # Calculate multiple scattering of aerosols for each band
    Lam = (Lam_11).multiply(((ESUN).divide(ESUNImg.select('B12')))).multiply((eps.multiply(ee.Image(-1))).multiply((bands_nm.divide(ee.Image(2190)))).exp());

    # diffuse transmittance
    trans = Tr.multiply(ee.Image(-1)).divide(ee.Image(2)).multiply(ee.Image(1).divide(cosdSatZe)).exp();

    # Compute water-leaving radiance
    Lw = Lrc.subtract(Lam).divide(trans);

    # water-leaving reflectance
    pw = (Lw.multiply(pi).multiply(d.pow(2)).divide(ESUN.multiply(cosdSunZe)));

    # remote sensing reflectance
    Rrs_S2B = (pw.divide(pi).arrayProject([0]).arrayFlatten([bands]).slice(0,9));

    # set negatives to null
    Rrs_S2B = Rrs_S2B.updateMask(Rrs_S2B.select('B1').gt(0)).multiply(mask)

    return Rrs_S2B.set('system:time_start',img.get('system:time_start'))

# Secchi Code

## Creating mask functions

These functions mask clouds based on the QA_PIXEL band (maskL8sr), select pixels that are >= 75% water (jrcMask), and a 30m buffer around roads to mask bridges (roadMask)

In [ ]:
AOI = ee.FeatureCollection("users/greeneji/MiCorp_Secchi_4ha_dissolve")
START_DATE = '2019-05-01'
END_DATE = '2023-08-30'
CLOUD_FILTER = 50
CLD_PRB_THRESH = 40
NIR_DRK_THRESH = 0.15
CLD_PRJ_DIST = 1
BUFFER = 50

#import S2 collection & join s2cloudless
def get_s2_sr_cld_col(aoi, start_date, end_date):

    # Import and filter s2cloudless.
    s2_cloudless_col = (ee.ImageCollection('COPERNICUS/S2_CLOUD_PROBABILITY')
        .filterBounds(aoi)
        .filterDate(start_date, end_date))

    # Import and filter s2 raw images.
    s2_raw_col = (ee.ImageCollection('COPERNICUS/S2_HARMONIZED')
        .filterBounds(aoi)
        .filterDate(start_date, end_date)
        .filter(ee.Filter.lte('CLOUDY_PIXEL_PERCENTAGE', CLOUD_FILTER)))

    # Join the filtered s2cloudless collection to the S2 collection by the 'system:index' property.
    return ee.ImageCollection(ee.Join.saveFirst('s2cloudless').apply(**{
        'primary': s2_raw_col,
        'secondary': s2_cloudless_col,
        'condition': ee.Filter.equals(**{
            'leftField': 'system:index',
            'rightField': 'system:index'
        })
    }))

s2_sr_cld_col_eval = get_s2_sr_cld_col(AOI, START_DATE, END_DATE)

#add SCL band from SR
s2_sr_col = (ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED')
    .filterBounds(AOI)
    .filterDate(START_DATE, END_DATE)
    .filter(ee.Filter.lte('CLOUDY_PIXEL_PERCENTAGE', CLOUD_FILTER)))

s2_sr_cld_col_eval = ee.ImageCollection.combine(s2_sr_cld_col_eval, s2_sr_col)

s2_sr_cld_col_eval = s2_sr_cld_col_eval.select('B1', 'B2', 'B3', 'B4', 'B5', 'B6', 'B7', 'B8', 'B8A', 'B9', 'B10', 'B11', 'B12','QA10', 'QA20', 'QA60', 'SCL')

#add cloud bands
def add_cloud_bands(img):
    # Get s2cloudless image, subset the probability band.
    cld_prb = ee.Image(img.get('s2cloudless')).select('probability')

    # Condition s2cloudless by the probability threshold value.
    is_cloud = cld_prb.gt(CLD_PRB_THRESH).rename('clouds')

    # Add the cloud probability layer and cloud mask as image bands.
    return img.addBands(ee.Image([cld_prb, is_cloud]))


def add_shadow_bands(img):
    # Identify water pixels from the SCL band.
    not_water = img.select('SCL').neq(6)

    # Identify dark NIR pixels that are not water (potential cloud shadow pixels).
    SR_BAND_SCALE = 1e4
    dark_pixels = img.select('B8').lt(NIR_DRK_THRESH*SR_BAND_SCALE).multiply(not_water).rename('dark_pixels')

    # Determine the direction to project cloud shadow from clouds (assumes UTM projection).
    shadow_azimuth = ee.Number(90).subtract(ee.Number(img.get('MEAN_SOLAR_AZIMUTH_ANGLE')));

    # Project shadows from clouds for the distance specified by the CLD_PRJ_DIST input.
    cld_proj = (img.select('clouds').directionalDistanceTransform(shadow_azimuth, CLD_PRJ_DIST*10)
        .reproject(**{'crs': img.select(0).projection(), 'scale': 100})
        .select('distance')
        .mask()
        .rename('cloud_transform'))

    # Identify the intersection of dark pixels with cloud shadow projection.
    shadows = cld_proj.multiply(dark_pixels).rename('shadows')

    # Add dark pixels, cloud projection, and identified shadows as image bands.
    return img.addBands(ee.Image([dark_pixels, cld_proj, shadows]))


def add_cld_shdw_mask(img):
    # Add cloud component bands.
    img_cloud = add_cloud_bands(img)

    # Add cloud shadow component bands.
    img_cloud_shadow = add_shadow_bands(img_cloud)

    # Combine cloud and shadow mask, set cloud and shadow as value 1, else 0.
    is_cld_shdw = img_cloud_shadow.select('clouds').add(img_cloud_shadow.select('shadows')).gt(0)

    # Remove small cloud-shadow patches and dilate remaining pixels by BUFFER input.
    # 20 m scale is for speed, and assumes clouds don't require 10 m precision.
    is_cld_shdw = (is_cld_shdw.focalMin(2).focalMax(BUFFER*2/20)
        .reproject(**{'crs': img.select([0]).projection(), 'scale': 20})
        .rename('cloudmask'))

    # Add the final cloud-shadow mask to the image.
    return img_cloud_shadow.addBands(is_cld_shdw)


def apply_cld_shdw_mask(img):
    # Subset the cloudmask band and invert it so clouds/shadow are 0, else 1.
    not_cld_shdw = img.select('cloudmask').Not()

    # Subset reflectance bands and update their masks, return the result.
    return img.select('B.*').updateMask(not_cld_shdw)

#add cloud mask
masked_coll = (s2_sr_cld_col_eval.map(add_cld_shdw_mask)
                             .map(apply_cld_shdw_mask))

In [ ]:
# jrc water occurrence mask
def jrcMask(image):
  jrc = ee.Image('JRC/GSW1_0/GlobalSurfaceWater')
  # select only water occurence
  occurrence = jrc.select('occurrence')
  # selectonly water occurences of greater than 75%
  water_mask = occurrence.mask(occurrence.gt(50))
  return image.updateMask(water_mask)

#Creating 30m road buffer mask
def roadMask(image):
  roads = ee.FeatureCollection("TIGER/2016/Roads")
  # 30m road buffer
  def bufferPoly30(feature):
    return feature.buffer(30)
  Buffer = roads.map(bufferPoly30)
    # Convert 'areasqkm' property from string to number.
  def func_uem(feature):
        num = ee.Number.parse(ee.String(feature.get('linearid')))
        return feature.set('linearid', num)
  roadBuffer = Buffer.map(func_uem)
  roadRaster = roadBuffer.reduceToImage(['linearid'], ee.Reducer.first())
  # create an image with a constant value of one to apply roadmask to
  blank = ee.Image.constant(1)
  inverseMask = blank.updateMask(roadRaster)
  # get reverse mask to have everything but roads kept
  mask = inverseMask.mask().Not()
  return image.updateMask(mask)

Create filters to select summer dates

In [ ]:
sum15 = ee.Filter.date('2015-05-01','2015-09-30')
sum16 = ee.Filter.date('2016-05-01','2016-09-30')
sum17 = ee.Filter.date('2017-05-01','2017-09-30')
sum18 = ee.Filter.date('2018-05-01','2018-09-30')
sum19 = ee.Filter.date('2019-05-01','2019-09-30')
sum20 = ee.Filter.date('2020-05-01','2020-09-30')
sum21 = ee.Filter.date('2021-05-01','2021-09-30')
sum22 = ee.Filter.date('2022-05-01','2022-09-30')
sum23 = ee.Filter.date('2023-05-01','2023-08-18')
Summers = ee.Filter.Or(sum15, sum16, sum17, sum18, sum19, sum20, sum21, sum22, sum23)

In [ ]:
#create point buffer for every pt
def wrap_buffer(pt_collection, radius):
  def bufferPoints(pt):
    pt_buffer = pt.buffer(radius).bounds()
    return pt_buffer
  return pt_collection.map(lambda ptfeat: bufferPoints(ptfeat))

ptBuffer = wrap_buffer(UniquePts, 60)

In [ ]:
# Filter landsat 8-9 collection by PATH and ROW
# geometry = ee.Geometry.Point([22.7876 ,44.16517])
geometry = ptBuffer
JRC = ee.Image("JRC/GSW1_3/GlobalSurfaceWater")
mask = JRC.select('occurrence').gt(50)

# oliCloudPerc = 50

target_image_number = 1

## Import collections


In [ ]:
# Import Collections w/ Sentinel-2
# MSI = ee.ImageCollection('COPERNICUS/S2_HARMONIZED')

ozone = ee.ImageCollection('TOMS/MERGED')

JRC = ee.Image('JRC/GSW1_1/GlobalSurfaceWater')
# Process
mask = JRC.select('occurrence').gt(0)

FC_S2A = masked_coll.filter(Summers) \
                .filterBounds(UniquePts) \
                .filterMetadata('SPACECRAFT_NAME', 'equals', 'Sentinel-2A') \
                .filterMetadata('CLOUDY_PIXEL_PERCENTAGE', "less_than", 50) \
                .map(jrcMask) \
                .map(roadMask) \
                .sort('system:time_start')

FC_S2B = masked_coll.filter(Summers) \
                .filterBounds(UniquePts) \
                .filterMetadata('SPACECRAFT_NAME', 'equals', 'Sentinel-2B') \
                .filterMetadata('CLOUDY_PIXEL_PERCENTAGE', "less_than", 50) \
                .map(jrcMask) \
                .map(roadMask) \
                .sort('system:time_start')

# print(FC_S2B.size().getInfo())
# print(FC_S2B.first().date().getInfo())

# merge s2a and s2b image collections
# FC_combined = FC_S2A.merge(FC_S2B).sort('system:time_start')

### Zonal Statistics by STORET ID

Creates a file for each STORET_ID.



convert Rrs to 32-bit integer

In [ ]:
from geemap.geemap import ee_to_pandas
import gc
out_dir = os.path.expanduser('~/Downloads')
if not os.path.exists(out_dir):
    os.makedirs(out_dir)

#set to first 5 for testing
STORETIDs1 = STORETIDs.iloc[0:, 1].dropna().to_list()

df1 = pd.DataFrame()

for station in STORETIDs1:
  #for yr in years:
    print(station)
    filteredBuffer = ee.FeatureCollection(ptBuffer.filter(ee.Filter.eq('STORETID', station)))

    #filter S2A by the filtered buffer and apply atm corr
    S2A_filt = FC_S2A.filterBounds(filteredBuffer)
    Rrs_S2A = S2A_filt.map(MAIN_S2A).sort('system:time_start')
    # print(Rrs_S2A.size().getInfo())

    Rrs_S2A_bands = Rrs_S2A \
      .select(['B1','B2','B3','B4','B5','B6','B7', 'B8', 'B8A'])

    # #filter S2B by the filtered buffer and apply atm corr
    S2B_filt = FC_S2B.filterBounds(filteredBuffer)
    Rrs_S2B = S2B_filt.map(MAIN_S2B).sort('system:time_start')

    Rrs_S2B_bands = Rrs_S2B \
      .select(['B1','B2','B3','B4','B5','B6','B7', 'B8', 'B8A'])

    #combine atm corr FCs
    Rrs_combined = Rrs_S2A_bands.merge(Rrs_S2B_bands).sort('system:time_start')
    # print(Rrs_combined.size().getInfo())

    out_landsat_stats = os.path.join(out_dir, 'MAIN_S2_lakes_' + str(station) + '_' + '.csv')
    zonalstats_out = geemap.zonal_statistics(Rrs_combined, filteredBuffer, out_landsat_stats, statistics_type='MEDIAN', scale=10, crs='EPSG:4326', return_fc=True)
    df2 = geemap.ee_to_geopandas(zonalstats_out)
    # df2 = df2.melt(['STORETID', 'COUNT_STOR', 'MEAN_Latit', 'MEAN_Longi'], var_name = "imageID", value_name = "values")
    df1 = pd.concat([df1,df2])

    print("Complete")

output_filepath = os.path.join(out_dir, 'MAIN_S2_secchilakes_stats.csv')
df1.to_csv(output_filepath)

10017
Computing statistics ...
Complete
10041
Computing statistics ...
Complete
10101
Computing statistics ...
Complete
10102
Computing statistics ...
Complete
10103
Computing statistics ...
Complete
10104
Computing statistics ...
Complete
10105
Computing statistics ...
Complete
10106
Computing statistics ...
Complete
10107
Computing statistics ...
Complete
10122
Computing statistics ...
Complete
20127
Computing statistics ...
Complete
20168
Computing statistics ...
Complete
30203
Computing statistics ...
Complete
30225
Computing statistics ...
Complete
30259
Computing statistics ...
Complete
30263
Computing statistics ...
Complete
40097
Computing statistics ...
Complete
50052
Computing statistics ...
Complete
50055
Computing statistics ...
Complete
50101
Computing statistics ...
Complete
50240
Computing statistics ...
Complete
80066
Computing statistics ...
Complete
80071
Computing statistics ...
Complete
80092
Computing statistics ...
Complete
80093
Computing statistics ...
Complete
